# 📦 Import Libraries
Import required Python packages.

In [3]:
import networkx as nx
import csv
import matplotlib.pyplot as plt

# 🏗️ Build Graph

Read CSV files and **build graphs**.
use default pass `data/following.csv` to create a graph


In [19]:
def build_graph(path="../data/following.csv"):
    try:
        with open(path, newline='', encoding='utf-8') as f:
            reader = csv.reader(f)
            # skip first line (source_id,relation,target_id)
            next(reader)
            G = nx.MultiDiGraph()

            for src, rel, tgt in reader:
                add_edge(G, src, tgt, rel)
    except Exception as e:
        raise RuntimeError(f"Unable to open file in this path: '{path}'") from e
    # return graph
    return G

default_attrs = {"type": "user", "color": "white", "size": "3.14"}
def add_edge(G, src, tgt, rel):
    if src not in G:
        G.add_node(src, **default_attrs)
    if tgt not in G:
        G.add_node(tgt, **default_attrs)
    G.add_edge(src, tgt, relation=rel)

# 🔍 Graph Type Analyzer
Detect if the graph is **directed** or **undirected**, and if it is **weighted** or **unweighted**.

In [1]:
def direct_status(G):
    return G.is_directed()
def weight_status(G):
    return nx.is_weighted(G)

# 🔢 Node Degree Calculator
Compute **in-degree** and **out-degree** of nodes in directed graphs,
or total degree in undirected graphs.

In [7]:
def degree_calculator(G):
    if G is None or G.number_of_nodes() == 0:
        print("Please load a graph first.")
        return

    if nx.is_directed(G):
        print("\n--- Calculate In-Degree And Out-Degree ---\n")
        choice = input("Do you want the degree of a specific node or all nodes (specific/all)? ").lower()
        if choice == "all":
            print("\n--- In-degree and Out-degree of all nodes ---")
            for node in G.nodes():
                print(f"Node {node}: In_Degree = {G.in_degree(node)}, Out_Degree = {G.out_degree(node)}")
        elif choice == "specific":
            node = input("Enter the desired node name: ")
            try:
                node = int(node)
            except ValueError:
                pass
            if G.has_node(node):
                print(f"Node {node}: In_Degree = {G.in_degree(node)}, Out_Degree = {G.out_degree(node)}")
            else:
                print(f"Node {node} doesn't exist in the graph.")
        else:
            print("Invalid choice.")
            return

    else:
        print("\n--- Calculate Degree ---\n")
        choice = input("Do you want the degree of a specific node or all nodes (specific/all)? ").lower()
        if choice == "all":
            print("\n--- Degree of all nodes ---")
            for node in G.nodes():
                print(f"Node {node}: Degree = {G.degree(node)}")
        elif choice == "specific":
            node = input("Enter the desired node name: ")
            try:
                node = int(node)
            except ValueError:
                pass
            if G.has_node(node):
                print(f"Node {node}: Degree = {G.degree(node)}")
            else:
                print(f"Node {node} doesn't exist in the graph.")
        else:
            print("Invalid choice.")
            return

# 🔗 Connected Components Finder
Detect **strongly connected components** in directed graphs
and **connected components** in undirected graphs .

In [8]:
def find_connected_components(G):
    if G is None or G.number_of_nodes() == 0:
        print("Please load a graph first.")
        return

    if nx.is_directed(G):
        print("\n--- Find Strongly Connected Components ---")
        strong_components = list(nx.strongly_connected_components(G))
        for i, component in enumerate(strong_components):
            print(f"Component {i+1}: {component}")

    else:
        print("\n--- Find Connected Components ---")
        connected_components = list(nx.connected_components(G))
        for i, component in enumerate(connected_components):
            print(f"Component {i+1}: {component}")

# 🧩 DFS and BFS Algorithm
---
Use **Depth-First Search (DFS)** to explore a graph deeply before backtracking,
and **Breadth-First Search (BFS)** to traverse it level by level.


In [14]:
def dfs(G, start_node, direction='out'):
    if direction != 'in' and direction != 'out' and direction != 'both':
        raise ValueError(f"Direction '{direction}' is not supported. Choose from 'in', 'out', or 'both'")
    stack = [start_node]
    visited = set()
    path = []
    while stack:
        node = stack.pop()
        if node in visited:
            continue
        visited.add(node)
        path.append(node)
        # graph is multy directed graph
        neighbors = list()
        if direction == 'out':
            neighbors = list(G.successors(node))
        elif direction == 'in':
            neighbors = list(G.predecessors(node))
        elif direction == 'both':
            out_neighbors = set(G.successors(node))
            in_neighbors = set(G.predecessors(node))
            neighbors = list(out_neighbors | in_neighbors)

        stack.extend(reversed(neighbors))
    return path

def bfs(G, start_node, direction='out'):
    if direction != 'in' and direction != 'out' and direction != 'both':
        raise ValueError(f"Direction '{direction}' is not supported. Choose from 'in', 'out', or 'both'")
    queue = [start_node]
    visited = set()
    path = []

    while queue:
        node = queue.pop()
        if node in visited:
            continue
        visited.add(node)
        path.append(node)
        neighbors = list()
        if direction == 'out':
            neighbors = list(G.successors(node))
        elif direction == 'in':
            neighbors = list(G.predecessors(node))
        elif direction == 'both':
            out_neighbors = set(G.successors(node))
            in_neighbors = set(G.predecessors(node))
            neighbors = list(out_neighbors | in_neighbors)
        queue = neighbors + queue
    return path

# 🛣️ Find the Shortest Path
Use **Dijkstra's algorithm** to compute the **shortest path** and **distance**
between two nodes in agraph.

In [9]:
def find_shortest_path(G, source, target):
    if G is None or G.number_of_nodes() == 0:
        print("Please load a graph first.")
        return

    print("\n--- Find shortest path ---")
    try:
        path = nx.dijkstra_path(G, source, target, weight='weight')
        distance = nx.dijkstra_path_length(G, source, target)
        print(f"\nThe shortest path between {source} and {target} is {path}")
        print(f"Path length: {distance}")
    except nx.NetworkXNoPath:
        print(f"No path exists between {source} and {target}.")


# 🌍 Centrality Analysis
Analyze node importance in the network using:
- Degree Centrality
- In/Out Degree (for directed graphs)
- Betweenness
- Closeness
- Eigenvector Centrality

In [10]:
def analyze_centrality(G):
    if G is None or G.number_of_nodes() == 0:
        print("Please load a graph first.")
        return

    print("\n--- Centrality Analysis ---")

    print("\n1. Degree Centrality (higher is more important):")
    degree_centrality = nx.degree_centrality(G)
    for node, centrality in sorted(degree_centrality.items(), key=lambda item: item[1], reverse=True)[:5]:
        print(f"- {node}: {centrality:.3f}")
    if nx.is_directed(G):
        print("\n   → In-Degree Centrality")
        in_degree_centrality = nx.in_degree_centrality(G)
        for node, centrality in sorted(in_degree_centrality.items(), key=lambda item: item[1], reverse=True)[:5]:
            print(f"- {node}: {centrality:.3f}")

        print("\n   → Out-Degree Centrality")
        out_degree_centrality = nx.out_degree_centrality(G)
        for node, centrality in sorted(out_degree_centrality.items(), key=lambda item: item[1], reverse=True)[:5]:
            print(f"- {node}: {centrality:.3f}")

    print("\n2. Betweenness Centrality (nodes on shortest paths):")
    betweenness_centrality = nx.betweenness_centrality(G)
    for node, centrality in sorted(betweenness_centrality.items(), key=lambda item: item[1], reverse=True)[:5]:
        print(f"- {node}: {centrality:.3f}")

    print("\n3. Closeness Centrality (average distance to all others):")
    closeness_centrality = nx.closeness_centrality(G)
    for node, centrality in sorted(closeness_centrality.items(), key=lambda item: item[1], reverse=True)[:5]:
        print(f"- {node}: {centrality:.3f}")

    print("\n4. Eigenvector Centrality (nodes on shortest paths):")